In [2]:
import numpy as np
import librosa, os, keras, time, scipy
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.utils import make_chunks
from collections import deque
import import_ipynb
#from Functions import *

In [3]:
dir1 = './IRMAS/TestingData/Part1/'
dir2 = './IRMAS/TestingData/Part2/'
dir3 = './IRMAS/TestingData/Part3/'
dir_array = [dir1,dir2,dir3]

instr = ['pia','gac','gel','vio','cel','sax','tru','cla','voi']
model = keras.models.load_model('ICmodel_solo.h5')

W1007 22:46:20.229534 18096 deprecation_wrapper.py:119] From C:\Users\kavra\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1007 22:46:21.435882 18096 deprecation.py:323] From C:\Users\kavra\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1007 22:46:24.567419 18096 deprecation_wrapper.py:119] From C:\Users\kavra\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [4]:
def to_categorical(list_final,instr):
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list[0]

# y_pred : vector of mean prediction for one track  [pia,gac,gel,vio,voi] 
# y_label: vector of mean ground truth for same one track
# returns difference accuracy for one track
def difference_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)): acc = acc + abs(y_pred[i]-y_label[i])
    acc = acc / len(y_pred)
    return 100*(1-acc)

# out of order
def existence_accuracy(y_pred,y_label):
    acc = 0
    for i in range(len(y_pred)):
        if (y_pred[i]>=thr[i] and y_label[i]==1): acc+=1
        elif (y_pred[i]<thr[i] and y_label[i]==0): acc+=1
    acc = acc / len(y_pred)
    return acc

def lrap_accuracy(y_pred,y_label,instr):
    z = instr.copy()
    y = y_pred.copy()
    y,z = (list(t) for t in zip(*sorted(zip(y,z))))
    
    chosen = []
    for i in range(len(instr)):
        if (y_label[i]==1): chosen.append(instr[i])
    
    sum = 0
    for i in range(len(instr)):
        num = 0
        den = 0
        if (y_label[i]==1):
            for j in range(len(instr)-1,-1,-1):
                den = den + 1
                if (z[j] in chosen): num = num + 1
                if (z[j] == instr[i]): break 
            sum = sum + num/den
    acc = sum / len(chosen)
    return [100*acc,sum,len(chosen)]

def auc_accuracy(auc,truth,instr):
    final = []
    for i in range(len(instr)):
        x0 = 0
        y0 = 0
        pos = 0
        neg = 0
    
        for j in truth[i]:
            if (j==1): pos = pos + 1
            else: neg = neg + 1

        a,t = (list(x) for x in zip(*sorted(zip(auc[i],truth[i]))))
        points = [[x0,y0]]
    
        for j in range (len(t)-1,-1,-1):
            if (t[j]): y0 = y0 + 1/pos
            else: x0 = x0 + 1/neg
            points.append([x0,y0])
        final.append(points)
    return final

def auc_plots(instr,auc_acc):
    ax = [plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2),
          plt.subplot2grid((2,6), (0,2), colspan=2),
          plt.subplot2grid((2,6), (0,4), colspan=2),
          plt.subplot2grid((2,6), (1,1), colspan=2),
          plt.subplot2grid((2,6), (1,3), colspan=2)]

    a = np.linspace(0,1,100)
    auc_acc = np.zeros(len(instr))
    for i in range(len(instr)):
        ax[i].title.set_text(instr[i])
        ax[i].plot(*zip(*auc_points[i][j]))
        ax[i].plot(a,a)
        auc_acc[i] = 1-np.trapz(*zip(*auc_points[i]))
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    
def fscore_accuracy(auc,truth):
    max_fscore = np.zeros(len(instr))
    max_thr = np.zeros(len(instr))

    for i in range(len(instr)):
        for thr in np.arange(0,1.001,0.001):
            tp,fp,fn,tn = 0,0,0,0
            for j in range(len(auc[i])):
                if (auc[i][j] >= thr and truth[i][j]==1): tp = tp + 1
                if (auc[i][j] >= thr and truth[i][j]==0): fp = fp + 1
                if (auc[i][j] <  thr and truth[i][j]==1): fn = fn + 1
                if (auc[i][j] <  thr and truth[i][j]==0): tn = tn + 1
            if ((tp==0 and fp==0) or (tn==0 and fn==0)): continue
            precision = tp / (tp+fp)
            recall = tn / (tn+fn)
            fscore = 2*precision*recall/(precision+recall)
            if (fscore > max_fscore[i]): 
                max_fscore[i] = fscore
                max_thr[i] = thr
    return max_fscore

In [5]:
def prediction(predict):
    prediction = model.predict(predict)
    s = np.zeros(len(prediction[0]))
    for pred in prediction:
        for i in range(len(pred)): s[i] = s[i] + pred[i]
    s[:] = [round(x/len(prediction),2) for x in s]
    return s

def predict_songs(start,end):
    auc = [[],[],[],[],[],[],[],[],[]]
    truth = [[],[],[],[],[],[],[],[],[]]
    mix_auc = [[],[],[],[],[],[],[],[],[]]
    mix_truth = [[],[],[],[],[],[],[],[],[]]
    solo_auc = [[],[],[],[],[],[],[],[],[]]
    solo_truth = [[],[],[],[],[],[],[],[],[]]
    sum_acc1,mix_acc1,solo_acc1 = 0,0,0
    sum_acc2,mix_acc2,solo_acc2 = 0,0,0
    len_acc2,len_mix2,len_solo2 = 0,0,0
    songs,mix_songs,solo_songs = 0,0,0
    it = 1
    start = 2*start
    end = 2*end - start
    text = True
    br = False
    unk_instr = False
    for dir in dir_array:
        for f in os.listdir(dir):
    
            if (start!=0):
                start = start -1
                continue
    
            if (unk_instr):
                unk_instr = False
                continue
        
            if (text):
                y_test = []
                x = open(dir+f)
                for line in x.readlines():
                    y_test.append(line[:3])
                    if (not (line[:3] in instr)): unk_instr = True
                if (unk_instr): continue
                y_label = to_categorical([y_test],instr)
        
            if not text:
                predict = []
                wav,sr = librosa.load(dir+f,mono=True,sr=22050)
                wav = wav / np.sqrt(np.mean(wav**2))
                chunks = []
                chunks = make_chunks(wav,sr)
                for ch in chunks[:-1]:
                    cqt = librosa.core.cqt(ch,n_bins=96,bins_per_octave=12,hop_length=256)
                    cqt = librosa.core.amplitude_to_db(np.abs(cqt))
                    predict.append(cqt)
                predict = np.expand_dims(np.array(predict),-1)
                y_pred = prediction(predict)
                
                acc1 = difference_accuracy(y_pred,y_label)
                [acc2,s,length] = lrap_accuracy(y_pred,y_label,instr)
                
                for i in range(len(instr)):
                    auc[i].append(y_pred[i])
                    truth[i].append(y_label[i])
                
                sum_acc1 = sum_acc1 + acc1
                sum_acc2 = sum_acc2 + s
                len_acc2 = len_acc2 + length
                songs = songs + 1
                
                if (sum(y_label)>1): 
                    mix_songs = mix_songs + 1
                    mix_acc1 = mix_acc1 + acc1
                    mix_acc2 = mix_acc2 + s
                    len_mix2 = len_mix2 + length
                    for i in range(len(instr)):
                        mix_auc[i].append(y_pred[i])
                        mix_truth[i].append(y_label[i])
                elif (sum(y_label)==1):
                    solo_songs = solo_songs + 1
                    solo_acc1 = solo_acc1 + acc1
                    solo_acc2 = solo_acc2 + s
                    len_solo2 = len_solo2 + length
                    for i in range(len(instr)):
                        solo_auc[i].append(y_pred[i])
                        solo_truth[i].append(y_label[i])
                
                #print("Predicted track    : ",f)
                #print("Expected output    : ", y_label)
                #print("Predicted output   : ", y_pred)
                #print("Difference Accuracy: ", round(acc1,2),"%")
                #print("Lrap Accuracy      : ", round(acc2,2),"%")
                #print("")
            
            if (it%400==0): 
                print(it,"songs examined.")
                print('Current Lrap',sum_acc2/len_acc2)
                print()
            
            if (it == end):
                br = True
                break
            it = it + 1
            text = not text
        if (br): break
    
    auc_points = auc_accuracy(auc,truth,instr)
    mix_auc_points = auc_accuracy(mix_auc,mix_truth,instr)
    solo_auc_points = auc_accuracy(solo_auc,solo_truth,instr)
    sum_acc2 = sum_acc2/len_acc2
    mix_acc2= mix_acc2/len_mix2
    solo_acc2 = solo_acc2/len_solo2
    
    return [[sum_acc1,mix_acc1,solo_acc1],[sum_acc2,mix_acc2,solo_acc2],[auc,mix_auc,solo_auc],[truth,mix_truth,solo_truth],[auc_points,mix_auc_points,solo_auc_points],[songs,mix_songs,solo_songs]]

In [ ]:
start = 1
end = start + 10000
[sum_acc1,sum_acc2,auc,truth,auc_points,songs] = predict_songs(start,end)

fscore0 = fscore_accuracy(auc[0],truth[0])
fscore1 = fscore_accuracy(auc[1],truth[1])
fscore2 = fscore_accuracy(auc[2],truth[2])

print("------------------------------------------------------")
print("ALL")
print("Average Difference Accuracy among",songs[0],"songs: ",round(sum_acc1[0]/songs[0],2),"%")
print("Average LRAP       Accuracy among",songs[0],"songs: ",round(100*sum_acc2[0],2),"%")
print()

print("MIX")
print("Average Difference Accuracy among",songs[1],"songs: ",round(sum_acc1[1]/songs[1],2),"%")
print("Average LRAP       Accuracy among",songs[1],"songs: ",round(100*sum_acc2[1],2),"%")
print()

print("SOLO")
print("Average Difference Accuracy among",songs[2],"songs: ",round(sum_acc1[2]/songs[2],2),"%")
print("Average LRAP       Accuracy among",songs[2],"songs: ",round(100*sum_acc2[2],2),"%")
print()

total_auc_acc = [0,0,0]
for j in range(0,3):
    for i in range(len(instr)): 
        auc_acc = 1-np.trapz(*zip(*auc_points[j][i]))
        #print("AUC for   ",instr[i],":",round(100*auc_acc,2),"%")
        #print("Fscore for",instr[i],":",round(100*fscore[i],2),"%")
        total_auc_acc[j] = auc_acc + total_auc_acc[j]
    
print()
print("ALL")
print("Total AUC:",round(100*total_auc_acc[0]/len(instr),2),"%")
print("Total Fscore:",round(100*np.average(fscore0),2),"%")

print()
print("MIX")
print("Total AUC:",round(100*total_auc_acc[1]/len(instr),2),"%")
print("Total Fscore:",round(100*np.average(fscore1),2),"%")

print()
print("SOLO")
print("Total AUC:",round(100*total_auc_acc[2]/len(instr),2),"%")
print("Total Fscore:",round(100*np.average(fscore2),2),"%")

400 songs examined.
Current Lrap 0.6696788194444445

800 songs examined.
Current Lrap 0.6801495814464119

